# Scraping the Unscrapable

Some sites are hard to scrape.

Sometimes you get blocked.
Sometimes the site is using a lot of fancy Javascript.

We'll see a few examples of methods we can use as workarounds for the former and introduce the tool Selenium that lets us automate dynamic interactions with the browser, which can help with the latter.

## How much is too much?

Sites have `robots.txt` pages that give guidelines about what they want to allow webcrawlers to access

In [ ]:
import requests

url = 'http://www.github.com/robots.txt'
response  = requests.get(url)
print(response.text)

Disallow: / means disallow everything (for all user-agents at the end that aren't covered earlier). Boxofficemojo is more accepting:

In [ ]:
url = 'http://www.boxofficemojo.com/robots.txt'
response  = requests.get(url)
print(response.text)

It's very common for sites to block you if you send too many requests in a certain time period. Sometimes all it takes to evade this is well-designed pauses in your scraping. 

2 general ways:
* pause after every request
* pause after each n requests

In [ ]:
#every request
import time

page_list = ['page1','page2','page3']

for page in page_list:
    ### scrape a website
    ### ...
    print(page)
    
    time.sleep(2)
    

In [ ]:
#every 200 requests
import time

page_list = ['page1','page2','page3','page4','page5','page6']

for i, page in enumerate(page_list):
    ### scrape a website
    ### ...
    print(page)
    
    if (i+1 % 200 == 0):
        time.sleep(320)

Or better yet, add a random delay (more human-like)

In [ ]:
import random

for page in page_list:
    ### scrape a website
    ### ...
    print(page)
    
    time.sleep(.5+2*random.random())

## How do I make requests look like a real browser?

In [ ]:
import sys
import requests
from bs4 import BeautifulSoup

url = 'http://www.reddit.com'

user_agent = {'User-agent': 'Mozilla/5.0'}
response  = requests.get(url, headers = user_agent)

We can generate a random user_agent

In [ ]:
from fake_useragent import UserAgent

ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

response  = requests.get(url, headers = user_agent)
print(response.text)

## Now to Selenium!

## What happens if I try to parse my gmail with `requests` and `BeautifulSoup`?

In [ ]:
import requests
from bs4 import BeautifulSoup

gmail_url="https://mail.google.com"
soup=BeautifulSoup(requests.get(gmail_url).text, "lxml")
print(soup.prettify())

Well, this is a tiny page. We get redirected. Soupifying this is useless, of course. Luckily, in this case we can see where we are sent to. In many of cases, you won't be so lucky. The page contents will be rendered by JavaScript by a browser, so just getting the source won't help you.

Anyway, let's follow the redirection for now.

In [ ]:
new_url = "https://mail.google.com/mail"

# get method will navigate the requested url.. 
soup =BeautifulSoup(requests.get(new_url).text)
print(soup.prettify())

In [ ]:
print(soup.find(id='Email'))

We have hit the login page. We can't get to the emails without logging in ... i.e. we need to actually interact with the browser using Selenium!

In [ ]:
pip install chromedriver_binary

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

#import chromedriver_binary


driver = webdriver.Chrome('/Users/adelweiss/Documents/Project/Project 2')
driver.get("https://mail.google.com")

# Alternatives to Chrome: Firefox, PhantomJS

### Interlude: how to include usernames and passwords

We are going to have to enter a username  and password in order to log in. However, we **don't** want to have our password uploaded to Github for people to scrape! One solution to this is to use _environment variables_.

In your directory, create a file called `.env` that has the following format:
```bash
USERNAME="your_username@gmail.com"
PASSWORD="your_password"
```
DON'T ADD THIS FILE TO GITHUB!
It is prudent to add a line `.env` to your `.gitignore`

We add two commands to the top of the cell:
```
%load_ext dotenv  # allows us to use the %dotenv "magic" command
%dotenv           # reads .env, and makes USERNAME and PASSWORD environment variables
```
We can now use `os.environ.get` to access the environment variables without having them appear in the notebook.

In [ ]:
!pip install python-dotenv

In [ ]:
# See notes about environment variables
%reload_ext dotenv
%dotenv
import os
EMAIL = os.environ.get('USERNAME')
PASSWORD = os.environ.get('PASSWORD')

# Show that this is working. Don't do this for PASSWORD!
print(EMAIL)

### Fill out username and password, hit enter to log in

Now let use this to log in.

In [ ]:
username_form = driver.find_element_by_id("identifierId")
username_form.send_keys(EMAIL) 

In [ ]:
username_form.send_keys(Keys.RETURN)

In [ ]:
password_form=driver.find_element_by_name("password") # note another approach
password_form.send_keys(PASSWORD) # enter password

In [ ]:
password_form.send_keys(Keys.RETURN)

### Click compose button to start a new email draft

In [ ]:
compose_button=driver.find_element_by_xpath('//div[text()="Compose"]')
compose_button.click()

### Write a nice, friendly (optional) message to your (least?) favorite person

In [ ]:
to_field = driver.find_element_by_name("to")
to_field.send_keys("email@gmail.com") # enter recipient email

In [ ]:
subject = driver.find_element_by_name("subjectbox")
subject.send_keys("This is an alert!")

In [ ]:
message_body = driver.find_element_by_xpath("//div[@aria-label='Message Body']")
message_body.send_keys("Hello,")
message_body.send_keys([Keys.RETURN, Keys.RETURN])
message_body.send_keys("I am a computer and I just became self aware!")

### Press the send button

In [ ]:
send_button = driver.find_element_by_xpath("//div[contains(@aria-label, 'Send')]")
send_button.click()

# Scraping Box Office Mojo with Selenium

In [ ]:
matrix_url = "http://www.boxofficemojo.com/movies/?id=matrix.htm"
driver.get(matrix_url)


In [ ]:
# 'contains' will find a match on the text, in this case return b tag
gross_selector = '//font[contains(text(), "Domestic")]/b'
print(driver.find_element_by_xpath(gross_selector).text)

In [ ]:
# scraping genre
genre_selector = '//a[contains(@href, "/genres/chart/")]/b'
for genre_anchor in driver.find_elements_by_xpath(genre_selector):
    print(genre_anchor.text)

In [ ]:
inf_adjust_2000_selector = '//select[@name="ticketyr"]/option[@value="2000"]'
driver.find_element_by_xpath(inf_adjust_2000_selector).click()

In [ ]:
go_button = driver.find_element_by_name("Go")
go_button.click()

Now the page has changed; it's showing inflation adjusted numbers. We can grab the new, adjusted number.

In [ ]:
gross_selector = '//font[contains(text(), "Domestic ")]/b'
print(driver.find_element_by_xpath(gross_selector).text)

# Scraping IMDB with Selenium

In [ ]:
url = "http://www.imdb.com"
driver.get(url)

In [ ]:
query = driver.find_element_by_id("navbar-query")
query.send_keys("Julianne Moore")

In [ ]:
query.send_keys(Keys.RETURN)

In [ ]:
name_selector = '//a[contains(text(), "Julianne Moore")]'
driver.find_element_by_xpath(name_selector).click()
current_url = driver.current_url

# Mixing Selenium and BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
"""Could use requests then send page.text to bs4
but Selenium actually stores the source as part of
the Selenium driver object inside driver.page_source

#import requests
#page = requests.get(current_url)
"""
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
soup.prettify()

In [ ]:
len(soup.find_all('a'))

In [1]:
driver.close()

NameError: name 'driver' is not defined

**Conclusion**: If a page is static, we can just use Beautiful Soup. If there is some dynamic component or interaction, we can then bring Selenium into the mix. Selenium can be used on its own or in conjunction with Beautiful Soup.

*References:* 

Documentation on finding elements:
- https://selenium-python.readthedocs.io/locating-elements.html

Xpath tutorial:
-  https://www.w3schools.com/xml/xpath_syntax.asp